# FP Growth


In [247]:
import pandas as pd
import numpy as np
import itertools
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import sys
import functools
sys.path.append('..')
from utils.utils import flatten, is_subset_of

In [2]:
data_path = '../data/store_data.csv'

In [556]:
class HT :
    def __init__(self):
        self.ht = None
    
    
    def build_ht(self, data, min_sup):
        if self.ht == None:
            self.ht = []
        keys, counts = np.unique(flatten(data), return_counts=True)
        ht_arr = np.array([(i,j) for i,j in zip(keys, counts)], dtype=[('key', int), ('counts', int)])
        rst = np.flip(np.sort(ht_arr, order='counts'))
        rst = filter(lambda x: x if x[1]>=min_sup else None, rst)
        rst = [ {'info':d, 'next': None}  for d in rst]
        self.ht = list(rst)
    
    def find_until_none(self, key, addr):
        for d in self.ht:
            if d['info'][0] == key and d['next'] != None:
                cur = d['next']
                while cur.next_instance != None:
                    cur = cur.next_instance
                cur.next_instance = addr
                return
            elif d['info'][0] == key and d['next'] == None:
                d['next'] = addr
                return
        raise "No wanted key it is wierd"
    
    def find_key_in_ht(self, key):
        for d in self.ht:
            if d['info'][0] == key:
                return d

In [577]:
class Node :
    def __init__(self, name):
        self.numOfChild = 0
        self._key = name
        self._count = 0
        self.next_instance = None
        self.slots = None
        self.parent = None
    
    @classmethod
    def DFS(self, cls):
        if cls.children == None:
            return
        for nkeys,value in cls.children.items():
            print(nkeys, value.count)
            Node.DFS(value)
        
    @property
    def key(self):
        return self._key
    
    @property
    def children(self):
        return self.slots
    
    @property
    def count(self):
        return self._count
    @count.setter
    def count(self, addOne):
        self._count += addOne
    
    def insertNode(self, node):
        if self.slots == None:
            self.slots = {}
        self.slots[node.key] = node
        node.parent = self
    
    def findWithKey(self, key):
        if self.slots == None:
            return None
        for nkey,value in self.slots.items():
            if nkey == key:
                return value
        return None

    def contain_with(self, key):
        if self.slots == None:
            return False
        for nkey,value in self.slots.items():
            if nkey == key:
                return True
        return False
    
    def __add__(self, other):
        self.insertNode(other)
        self.numOfChild += 1
        other.count = 1
        return other

    def __gt__(self, other_key):
        return self.findWithKey(other_key)
        

In [583]:
root = Node('root')
root + Node(55)
(root > 55) + Node(77)
child = (root > 55) > 77
print(child.parent.key)
print(child.parent.parent.key)
c = (root > 55) + Node(56)
print(c.parent.key)

55
root
55


# Unit test of Node

In [558]:
class FPEstimator(HT):
    ''' @FPEstimator inherented from HT
        @pro:
            1. data
            2. shape
            3. l1label
            4. Encoder
    '''
    def __init__(self, df_data):
        super().__init__()
        self.df_data = df_data
        self.dshape = df_data.shape
        self.le = LabelEncoder()
        self.extract_items(df_data)
        self.root = None
        
    @property
    def data(self):
        return self.df_data
    @property
    def shape(self):
        return self.dshape
    
    @property
    def l1label(self):
        return list(zip(self.itemset, self.l1_count))
    @property
    def Encoder(self):
        return self.le     
    
    def extract_items(self, data):
        self.raw_items = []
        for index, row in data.iterrows():
            row = pd.Series(row).dropna().values
            self.raw_items.append(np.array(row).tolist())
        self.itemset, self.l1_count = np.unique(flatten(self.raw_items), return_counts=True)
        self.le.fit(self.itemset)
        self.encoded_data = [ self.le.transform(d) for d in self.raw_items ]
    
    def order_cmp(self,x,y):
        return self.htl.index(x) - self.htl.index(y)
    
    def build_tree(self, min_sup):
        if self.root == None:
            self.root = Node('r')
        self.build_ht(self.encoded_data, min_sup)
        cur = None
        self.rearr_data = []
        htk = { d['info'][0] for d in self.ht }
        self.htl = [ d['info'][0] for d in self.ht ]
        for index, row in enumerate(self.encoded_data):
            l = list(set(row).intersection(htk))
            l = sorted(l,key=functools.cmp_to_key(self.order_cmp))
            self.rearr_data.append(l)
            
        for index, row in enumerate(self.rearr_data):
            cur = self.root
            for d in row:
                print(d, end=' ')
                if not cur.contain_with(d):
                    cur = cur + Node(d)
                    self.find_until_none(d, cur)
                else:
                    cur = cur.findWithKey(d)
                    cur.count = 1
        self.min_sup = min_sup

    def gen_fis(self)
        if not hasattr(self, 'min_sup'):
            raise Exception(" The tree is not built in advanced. ")
        lk = [ d['info'] for d in self.ht ]
        
                      
        

In [559]:
data_df = pd.DataFrame([['a','c','d','f','g','i','m','p'], ['a','b','c','f','i','m','o'], ['b','f', 'h', 'j','o'],['b','c','k','s','p'], ['a','c','e','f','l','m','n','p'] ])

In [560]:
data_df.head()

,0,1,2,3,4,5,6,7
0,a,c,d,f,g,i,m,p
1,a,b,c,f,i,m,o,None
2,b,f,h,j,o,None,None,None
3,b,c,k,s,p,None,None,None
4,a,c,e,f,l,m,n,p


In [561]:
fpe = FPEstimator(data_df)


In [562]:
fpe.encoded_data

[array([ 0,  2,  3,  5,  6,  8, 12, 15]),
 array([ 0,  1,  2,  5,  8, 12, 14]),
 array([ 1,  5,  7,  9, 14]),
 array([ 1,  2, 10, 16, 15]),
 array([ 0,  2,  4,  5, 11, 12, 13, 15])]

In [563]:
fpe.l1label

[('a', 3),
 ('b', 3),
 ('c', 4),
 ('d', 1),
 ('e', 1),
 ('f', 4),
 ('g', 1),
 ('h', 1),
 ('i', 2),
 ('j', 1),
 ('k', 1),
 ('l', 1),
 ('m', 3),
 ('n', 1),
 ('o', 2),
 ('p', 3),
 ('s', 1)]

In [584]:
fpe.ht

[{'info': (5, 4), 'next': <__main__.Node at 0x7fab4128cf28>},
 {'info': (2, 4), 'next': <__main__.Node at 0x7fab4128c160>},
 {'info': (15, 3), 'next': <__main__.Node at 0x7fab4128cb38>},
 {'info': (12, 3), 'next': <__main__.Node at 0x7fab4128c470>},
 {'info': (1, 3), 'next': <__main__.Node at 0x7fab4128cf98>},
 {'info': (0, 3), 'next': <__main__.Node at 0x7fab4128cda0>}]

In [565]:
fpe.build_tree(3)

5 2 15 12 0 5 2 12 1 0 5 1 2 15 1 5 2 15 12 0 

In [566]:
fpe.root

In [567]:
Node.DFS(fpe.root)

5 4
2 3
15 2
12 2
0 2
12 1
1 1
0 1
1 1
2 1
15 1
1 1


In [568]:
print(fpe.ht)

[{'info': (5, 4), 'next': <__main__.Node object at 0x7fab4128cf28>}, {'info': (2, 4), 'next': <__main__.Node object at 0x7fab4128c160>}, {'info': (15, 3), 'next': <__main__.Node object at 0x7fab4128cb38>}, {'info': (12, 3), 'next': <__main__.Node object at 0x7fab4128c470>}, {'info': (1, 3), 'next': <__main__.Node object at 0x7fab4128cf98>}, {'info': (0, 3), 'next': <__main__.Node object at 0x7fab4128cda0>}]


In [572]:
for d in fpe.ht:
    cur = d['next']
    while cur != None:
        print(cur.key, cur.count)
        cur = cur.next_instance
    

5 4
2 3
2 1
15 2
15 1
12 2
12 1
1 1
1 1
1 1
0 2
0 1
